## 회귀분석
- 회귀 분석은 실젯값과 예측값의 차이를 기반으로 한 지표들을 중심으로 성능평가 지표가 발전
- 실젯값과 예측값의 차이를 구해서 이것들을 단순히 합하게 되면 +와 -가 섞여 오류를 상쇄할 수 있다.

### MAE
- 실젯값과 예측값의 차이를 절댓값으로 변환하여 평균한 것
- 에러의 크기가 그대로 반영
- 이상치에 영향을 받는다

### MSE
- 실젯값과 예측값의 차이를 제곱해 평균한 것
- 실젯값과 예측값 차이의 면적 합을 의미
- 특이값이 존재하면 수치가 증가

### RMSE
- 실젯값과 예측값의 차이를 제곱해 평균한 것에 루트
- 에러에 제곱을 하면 에러가 클수록 그에 따른 가중치가 높이 반영
- 손실이 기하급수적으로 증가하는 상황에서 실제 오류 평균보다 값이 더 커지는 것을 상쇄하기 위해 사용

### MLSE
- 실젯값과 예측값의 차이를 제곱해 평균한 것에 로그를 적용한 것
- RMSE와 같이 손실이 기하급수적으로 증가하는 상황에서 실제 오류 평균보다 값이 더 커지는 것을 상쇄하기 위해 사용

### MAPE
- MAE를 퍼센트로 표시
- 오차가 예측값에서 차지하는 정도를 나타냄

### 회귀분석의 순서
1. 데이터를 확인
2. 데이터를 분할
3. 데이터의 전처리
4. 모델 학습
5. 성능평가 및 예측값 저장